In [34]:
import os as os
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import wget as w

In [24]:
# assing the folder for storing files
data_dir = '/home/artur/mlops-zoomcamp/notebooks/data'
os.chdir(data_dir)
os.getcwd()

'/home/artur/mlops-zoomcamp/notebooks/data'

In [48]:

# !wget 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'
# !wget 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet'

--2024-05-14 09:26:09--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.172.99.3, 18.172.99.89, 18.172.99.122, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.172.99.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-02.parquet.1’

yellow_tripdata_202 100%[===================>]  45.54M   128MB/s    in 0.4s    

2024-05-14 09:26:10 (128 MB/s) - ‘yellow_tripdata_2023-02.parquet.1’ saved [47748012/47748012]



In [45]:
for i in os.listdir():
    if i not in ('yellow_tripdata_2023-01.parquet', 'yellow_tripdata_2023-02.parquet'):
        os.remove(i)

In [65]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
    
    print(filename, ' ', 'shape',' ', df.shape)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    
    print(df.duration.std())
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    print(f'std:  {df.duration.std():.2f}')
    # df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
       
    return df

In [66]:
df_train = read_dataframe('yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('yellow_tripdata_2023-02.parquet')

yellow_tripdata_2023-01.parquet   shape   (3066766, 19)
0 days 00:42:35.661074
std:  42.59
yellow_tripdata_2023-02.parquet   shape   (2913955, 19)
0 days 00:42:50.526105
std:  42.84


In [70]:
# Share left
round(df_train[(df_train.duration >= 1) & (df_train.duration <= 60)].shape[0]/df_train.shape[0],2)

0.98

In [71]:
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)]
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [72]:
categorical = ['PULocationID', 'DOLocationID']
numerical = []#'trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [75]:
X_train.shape[1]

515

In [74]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [76]:
lr = LinearRegression()
lr.fit(X_train, y_train)
mean_squared_error(y_train, lr.predict(X_train), squared=False)

7.649261961655295

In [77]:
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.811819038564729

In [16]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)